LTSM


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import pickle
import os
import mlflow
import mlflow.tensorflow
import time
import json 

#usefull functions:
def create_sequences_by_scenario(df, seq_length, feature_cols, target_col):
    X, y, seq_scenario_ids = [], [], []
    for scenario_id, group in df.groupby('scenario_id'):
        group = group.sort_index()
        data = group[feature_cols + [target_col]].values
        if len(data) <= seq_length:
            continue
        for i in range(seq_length, len(data)):
            X.append(data[i-seq_length:i, :-1])
            y.append(data[i, -1])
            seq_scenario_ids.append(scenario_id)
    return np.array(X), np.array(y), np.array(seq_scenario_ids)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # Avoid division by zero
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))
# Load data
feature_and_target ={}
version = '0.0.1'
os.makedirs(f'./model_trained/LSTM_{version}',exist_ok=True)
root = './dataset/junctions/'
metrics_df = pd.DataFrame(columns=['junction', 'MAE', 'RMSE', 'MAPE', 'SMAPE', 'R2', 'Peak_Error'])
mlflow.set_tracking_uri("file:model_trained/mlruns")

seq_length = 23
n_features = 0
n_targets = 1
with mlflow.start_run(run_name=f"LSTM_{version}"):

        # Log parameters
    mlflow.log_param("seq_length", seq_length)
    mlflow.log_param("n_targets", n_targets)
    mlflow.log_param("epochs", 50)
    mlflow.log_param("batch_size", 32)

    for filename in os.listdir(root):
        df = pd.read_parquet(os.path.join(root, filename))
        raw_cols = [col for col in df.columns]
        junction = os.path.splitext(filename)[0]
        df[f'{junction}_lag1'] = df[junction].shift(1)
        # df[f'{junction}_lag7'] = df[junction].shift(7)
        # df[f'{junction}_roll_mean_7'] = df[junction].rolling(window=7).mean()

        # Drop rows with NaN
        df = df.fillna(0)

        # Features and targets
        target = junction
        features = [col for col in df.columns if col != target]
        feature_and_target[junction] = {
            "target": target,
            "features": features,
            "raw_cols": raw_cols
        }


        feature_scaler = MinMaxScaler()
        target_scaler = MinMaxScaler()

        scaled_features = feature_scaler.fit_transform(df[features])
        scaled_target = target_scaler.fit_transform(df[[target]])

        # Save the scalers for later use
        with open(f'model_trained/LSTM_{version}/{junction}_feature_scaler.save', 'wb') as f:
            pickle.dump(feature_scaler, f)

        with open(f'model_trained/LSTM_{version}/{junction}_target_scaler.save', 'wb') as f:
            pickle.dump(target_scaler, f)


        # # Log dataset file names, number of scenarios, rows, and columns
        # mlflow.log_param("num_scenarios", len(scenario_ids))
        # mlflow.log_param("num_rows", df.shape[0])
        # mlflow.log_param("num_features", len(features))

        scaled_df = pd.DataFrame(np.hstack((scaled_features, scaled_target)), columns=features + [target])
        scaled_df['scenario_id'] = df['scenario_id'].values

        X, y, seq_scenario_ids = create_sequences_by_scenario(scaled_df, seq_length, features, target)

        scenario_ids = df['scenario_id'].unique()
        split = int(0.8 * len(scenario_ids))
        train_scenarios = scenario_ids[:split]
        test_scenarios = scenario_ids[split:]

        X_train = X[np.isin(seq_scenario_ids, train_scenarios)]
        y_train = y[np.isin(seq_scenario_ids, train_scenarios)]
        X_test = X[np.isin(seq_scenario_ids, test_scenarios)]
        y_test = y[np.isin(seq_scenario_ids, test_scenarios)]

        n_features = len(features)
        mlflow.log_param(f"n_features_{junction}", n_features)
        n_targets = 1

        # Build multi-output LSTM model
        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(50, activation='relu', input_shape=(seq_length, n_features)),
            tf.keras.layers.Dense(n_targets)
        ])
        
        model.compile(optimizer='adam', loss='mse')
        

        # Train
        start = time.time()

        history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

        elapsed_sec = time.time() - start
        minutes, rem = divmod(elapsed_sec, 60)
        seconds, milliseconds = divmod(rem, 1)
        milliseconds = int(milliseconds * 1000)
        formatted_time = f"{int(minutes)}m:{int(seconds)}s:{milliseconds}ms"
        mlflow.log_param(f"{junction}_training_time", formatted_time)

        # Save the model
        model.save(f'model_trained/LSTM_{version}/{junction}.h5')

        model_summary = []
        model.summary(print_fn=lambda x: model_summary.append(x))
        mlflow.log_text(
            "\n".join(model_summary),
            artifact_file=f"{junction}/model_summary.txt"
        )

        # Log training and validation loss curves
        plt.plot(history.history['loss'], label='train_loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.legend(); plt.title('Loss over epochs')
        plt.savefig(f'model_trained/LSTM_{version}/{junction}_loss_curve.png')
        mlflow.log_artifact(
            f'model_trained/LSTM_{version}/{junction}_loss_curve.png',
            artifact_path=f"{junction}/plots"
        )

        # Predict & inverse scale
        y_pred = model.predict(X_test)
        y_test_inv = target_scaler.inverse_transform(y_test.reshape(-1, 1))
        y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1, 1))

        # Simple evaluation per node
        for i in range(y_test_inv.shape[1]):
            mae = mean_absolute_error(y_test_inv[:, i], y_pred_inv[:, i])
            print(f'MAE: {mae:.4f}')


        mae = mean_absolute_error(y_test_inv, y_pred_inv)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
        smape_val = smape(y_test_inv, y_pred_inv)
        r2 = r2_score(y_test_inv, y_pred_inv)
        peak_err = peak_error(y_test_inv, y_pred_inv)

        metrics_df = pd.concat([metrics_df, pd.DataFrame({
            'junction': [junction],
            'MAE': [mae],
            'RMSE': [rmse],
            'MAPE': [mape],
            'SMAPE': [smape_val],
            'R2': [r2],
            'Peak_Error': [peak_err]
        })], ignore_index=True)

            # Log metrics
        mlflow.log_metrics({
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape,
            "SMAPE": smape_val,
            "R2": r2,
            "Peak_Error": peak_err
        })
        mlflow.log_artifact(
            f'model_trained/LSTM_{version}/{junction}_feature_scaler.save',
            artifact_path=f"{junction}/scalers"
        )
        mlflow.log_artifact(
            f'model_trained/LSTM_{version}/{junction}_target_scaler.save',
            artifact_path=f"{junction}/scalers"
        )
        input_example = np.random.rand(1, seq_length, n_features)
        mlflow.tensorflow.log_model(
            model,
            artifact_path=junction,       # dossier interne pour stocker les fichiers
            registered_model_name=junction, # nom du modèle dans MLflow
            input_example=input_example
        )


metrics_df.set_index('junction', inplace=True)
for metric in metrics_df.columns:
    plt.figure(figsize=(12,6))
    metrics_df[metric].plot(kind='bar', color='skyblue')
    plt.title(f'LSTM Model {metric} Comparison Across Junctions')
    plt.ylabel(metric)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plot_path = f'model_trained/LSTM_{version}/{metric}_comparison.png'
    plt.savefig(plot_path)
    plt.close()
    
    # log the plot to MLflow
    mlflow.log_artifact(plot_path)
with open("feature_and_target.json", "w") as f:
    json.dump(feature_and_target, f, indent=4)

2025-08-24 12:03:51.542006: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-24 12:03:51.870931: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-24 12:03:52.239323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756033432.548534   43681 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756033432.648210   43681 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756033433.350599   43681 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 76ms/step - loss: 37638.3867 - val_loss: 218634.2188
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 50042.9297 - val_loss: 59047.2812
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 13630.9502 - val_loss: 234.5295
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 158.8276 - val_loss: 417.4804
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 377.4199 - val_loss: 50.7521
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 122.7127 - val_loss: 162.9412
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 170.8882 - val_loss: 8.5410
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 19.9097 - val_loss: 65.8870
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 19.8858 - val_loss: 1.8814
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 8.8540 - val_loss: 2.7085
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 13.7027 - val_loss: 1.5567
Epoch 12/50
25/25 ━━━━━

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step


/tmp/ipykernel_43681/1802571160.py:183: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, pd.DataFrame({
2025/08/24 12:05:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


MAE: 0.0046
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


...

Epoch 1/50


/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 64911.6172 - val_loss: 163083.8438
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 20375.7871 - val_loss: 29100.2969
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 4386.8521 - val_loss: 2861.4824
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 6942.5288 - val_loss: 18085.7910
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3902.7686 - val_loss: 150243.8594
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 8117.3286 - val_loss: 33845.2539
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 4024.3823 - val_loss: 1623.5437
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 971.1032 - val_loss: 3.0783
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 137.6329 - val_loss: 114.8189
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 237.6510 - val_loss: 2.6388
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 84.6742 - val_loss: 3.6082
Epoch 12/50
25

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step
MAE: 0.0013


2025/08/24 13:24:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x703b08ddff50>>
Traceback (most recent call last):
  File "/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
scaled_df.head()

,scenario_id,time_id,J434_elevation,P298_diameter,P301_diameter,P298_initial_status,P301_initial_status,P298_length,P301_length,P298_minor_loss,P301_minor_loss,P298_roughness,P301_roughness,J434_lag1,J434
0,1.0,0.0,0.355601,0.318596,0.318596,0.0,0.0,0.304077,0.072027,0.0,0.0,0.465571,0.836645,0.667289,0.028461
1,2.0,0.0,0.809477,0.068133,0.068133,0.0,0.0,0.679583,0.064813,0.0,0.0,0.100005,0.060956,0.028461,0.245149
2,3.0,0.0,0.821379,0.356280,0.356280,0.0,0.0,0.948148,0.919407,0.0,0.0,0.036234,0.367950,0.245149,0.622423
3,4.0,0.0,0.245046,0.410557,0.410557,0.0,0.0,0.176097,0.338755,0.0,0.0,0.881240,0.274330,0.622423,0.770930
4,5.0,0.0,0.582206,0.319809,0.319809,0.0,0.0,0.437278,0.330571,0.0,0.0,0.111404,0.184478,0.770930,0.864857


LightGBM Model

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import os
import mlflow
import json
import time

# -----------------------------
# Metrics
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = './dataset/junctions/'
version = '0.0.1'
os.makedirs(f'./model_trained/LightGBM_{version}', exist_ok=True)
mlflow.set_tracking_uri("file:model_trained/mlruns")

metrics_df = pd.DataFrame(columns=['junction', 'MAE', 'RMSE', 'MAPE', 'SMAPE', 'R2', 'Peak_Error'])
feature_and_target = {}

lag_steps = [1,2,3]  

# lists to store global predictions
global_y_true, global_y_pred = [], []
with mlflow.start_run(run_name=f"LightGBM_{version}"):

    for filename in os.listdir(root)[:70]:
        df = pd.read_parquet(os.path.join(root, filename))
        junction = os.path.splitext(filename)[0]

        # -----------------------------
        # Create lag features
        # -----------------------------
        for lag in lag_steps:
            df[f'{junction}_lag{lag}'] = df[junction].shift(lag)

        # Fill NaNs with zero
        df = df.fillna(0)

        features = [col for col in df.columns if col != junction]
        target = junction
        feature_and_target[junction] = {"target": target, "features": features,'lags':lag_steps }

        # -----------------------------
        # Scaling
        # -----------------------------
        feature_scaler = MinMaxScaler()
        target_scaler = MinMaxScaler()

        df[features] = feature_scaler.fit_transform(df[features])
        df[[target]] = target_scaler.fit_transform(df[[target]])

        # Save scalers
        os.makedirs(f'model_trained/LightGBM_{version}/scalers', exist_ok=True)
        with open(f'model_trained/LightGBM_{version}/scalers/{junction}_feature_scaler.save', 'wb') as f:
            pickle.dump(feature_scaler, f)
        with open(f'model_trained/LightGBM_{version}/scalers/{junction}_target_scaler.save', 'wb') as f:
            pickle.dump(target_scaler, f)

        # -----------------------------
        # Train/test split by scenario_id
        # -----------------------------
        scenario_ids = df['scenario_id'].unique()
        split = int(0.8 * len(scenario_ids))
        train_scenarios = scenario_ids[:split]
        test_scenarios = scenario_ids[split:]

        train_df = df[df['scenario_id'].isin(train_scenarios)]
        test_df = df[df['scenario_id'].isin(test_scenarios)]

        X_train, y_train = train_df[features], train_df[target]
        X_test, y_test = test_df[features], test_df[target]

        # -----------------------------
        # Train LightGBM Regressor
        # -----------------------------
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.01,
            'num_leaves': 31,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1
        }

        start_time = time.time()
        model = lgb.train(
            params,
            train_data,
            num_boost_round=5000,
            valid_sets=[train_data, valid_data],
            callbacks=[
                lgb.early_stopping(stopping_rounds=100),
                lgb.log_evaluation(period=100)
            ]
        )
        elapsed_sec = time.time() - start_time
        print(f"{junction} training time: {elapsed_sec:.2f} sec")

        # -----------------------------
        # Prediction & evaluation
        # -----------------------------
        y_pred = model.predict(X_test, num_iteration=model.best_iteration)
        y_test_inv = target_scaler.inverse_transform(y_test.values.reshape(-1,1))
        y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))

        # store for global metrics
        global_y_true.append(y_test_inv)
        global_y_pred.append(y_pred_inv)

        mae = mean_absolute_error(y_test_inv, y_pred_inv)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
        smape_val = smape(y_test_inv, y_pred_inv)
        r2 = r2_score(y_test_inv, y_pred_inv)
        peak_err = peak_error(y_test_inv, y_pred_inv)

        metrics_df = pd.concat([metrics_df, pd.DataFrame({
            'junction': [junction],
            'MAE': [mae],
            'RMSE': [rmse],
            'MAPE': [mape],
            'SMAPE': [smape_val],
            'R2': [r2],
            'Peak_Error': [peak_err]
        })], ignore_index=True)

        # Save model
        model.save_model(f'model_trained/LightGBM_{version}/{junction}.txt')

    # -----------------------------
    # Global metrics across all junctions
    # -----------------------------
    all_y_true = np.vstack(global_y_true)
    all_y_pred = np.vstack(global_y_pred)

    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'MAPE': mean_absolute_percentage_error(all_y_true, all_y_pred),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2': r2_score(all_y_true, all_y_pred),
        'Peak_Error': peak_error(all_y_true, all_y_pred)
    }

    print("\n✅ Global Metrics:")
    for k,v in global_metrics.items():
        print(f"{k}: {v:.4f}")
    mlflow.log_metrics(global_metrics)
    
# Save feature/target mapping
with open(f'model_trained/LightGBM_{version}/feature_and_target.json', 'w') as f:
    json.dump(feature_and_target, f, indent=4)

print("✅ LightGBM training completed")


/home/zayd/.config/matplotlib is not a writable directory
Matplotlib created a temporary cache directory at /tmp/matplotlib-k2_esvgw because there was an issue with the default path (/home/zayd/.config/matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.236709	valid_1's rmse: 0.241072
[200]	training's rmse: 0.229331	valid_1's rmse: 0.236764
[300]	training's rmse: 0.225019	valid_1's rmse: 0.235763
[400]	training's rmse: 0.221995	valid_1's rmse: 0.235552
[500]	training's rmse: 0.219323	valid_1's rmse: 0.235457
[600]	training's rmse: 0.217026	valid_1's rmse: 0.235466
Early stopping, best iteration is:
[553]	training's rmse: 0.218071	valid_1's rmse: 0.235411
J434 training time: 6.40 sec


/tmp/ipykernel_4136/372010587.py:144: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, pd.DataFrame({


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.241274	valid_1's rmse: 0.244168
[200]	training's rmse: 0.235198	valid_1's rmse: 0.241447
[300]	training's rmse: 0.231663	valid_1's rmse: 0.240862
[400]	training's rmse: 0.228899	valid_1's rmse: 0.240889
Early stopping, best iteration is:
[333]	training's rmse: 0.230696	valid_1's rmse: 0.240827
J246 training time: 5.75 sec
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.237435	valid_1's rmse: 0.238924
[200]	training's rmse: 0.230412	valid_1's rmse: 0.235455
[300]	training's rmse: 0.226454	valid_1's rmse: 0.234866
[400]	training's rmse: 0.223183	valid_1's rmse: 0.23461
Early stopping, best iteration is:
[385]	training's rmse: 0.223633	valid_1's rmse: 0.234589
J82 training time: 3.89 sec
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.227363	valid_1's rmse: 0.228167
[200]	training's rmse: 0.219543	valid_1's rmse: 0.22368
[300]

KeyboardInterrupt: 

LightGBM with engineered features (e.g., lags, moving averages) 

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pickle
import os
import mlflow
import json
import time

# -----------------------------
# Metrics
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = './dataset/junctions/'
version = '0.0.2_moving_avg'
os.makedirs(f'./model_trained/LightGBM_{version}', exist_ok=True)
mlflow.set_tracking_uri("file:model_trained/mlruns")

metrics_df = pd.DataFrame(columns=['junction', 'MAE', 'RMSE', 'MAPE', 'SMAPE', 'R2', 'Peak_Error'])
feature_and_target = {}

lag_steps = [1,2,3]
rolling_windows = [3,6,12]  # moving averages (in timesteps)

# lists to store global predictions
global_y_true, global_y_pred = [], []

with mlflow.start_run(run_name=f"LightGBM_{version}"):

    for filename in os.listdir(root)[:70]:
        df = pd.read_parquet(os.path.join(root, filename))
        junction = os.path.splitext(filename)[0]

        # -----------------------------
        # Lag features
        # -----------------------------
        for lag in lag_steps:
            df[f'{junction}_lag{lag}'] = df[junction].shift(lag)

        # -----------------------------
        # Moving average features
        # -----------------------------
        for w in rolling_windows:
            df[f'{junction}_rollmean{w}'] = df[junction].rolling(window=w, min_periods=1).mean()

        df = df.fillna(0)

        features = [col for col in df.columns if col != junction]
        target = junction
        feature_and_target[junction] = {"target": target, "features": features, 'lags':lag_steps ,'rolling':rolling_windows}

        # -----------------------------
        # Scaling
        # -----------------------------
        feature_scaler = MinMaxScaler()
        target_scaler = MinMaxScaler()
        df[features] = feature_scaler.fit_transform(df[features])
        df[[target]] = target_scaler.fit_transform(df[[target]])

        # Save scalers
        os.makedirs(f'model_trained/LightGBM_{version}/scalers', exist_ok=True)
        with open(f'model_trained/LightGBM_{version}/scalers/{junction}_feature_scaler.save', 'wb') as f:
            pickle.dump(feature_scaler, f)
        with open(f'model_trained/LightGBM_{version}/scalers/{junction}_target_scaler.save', 'wb') as f:
            pickle.dump(target_scaler, f)

        # -----------------------------
        # Train/test split by scenario_id
        # -----------------------------
        scenario_ids = df['scenario_id'].unique()
        split = int(0.8 * len(scenario_ids))
        train_scenarios = scenario_ids[:split]
        test_scenarios = scenario_ids[split:]
        train_df = df[df['scenario_id'].isin(train_scenarios)]
        test_df = df[df['scenario_id'].isin(test_scenarios)]

        X_train, y_train = train_df[features], train_df[target]
        X_test, y_test = test_df[features], test_df[target]

        # -----------------------------
        # Train LightGBM
        # -----------------------------
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.01,
            'num_leaves': 31,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1
        }
        start_time = time.time()
        model = lgb.train(
            params,
            train_data,
            num_boost_round=5000,
            valid_sets=[train_data, valid_data],
            callbacks=[
                lgb.early_stopping(stopping_rounds=100),
                lgb.log_evaluation(period=100)
            ]
        )

        # -----------------------------
        # Prediction & evaluation
        # -----------------------------
        y_pred = model.predict(X_test, num_iteration=model.best_iteration)
        y_test_inv = target_scaler.inverse_transform(y_test.values.reshape(-1,1))
        y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))

        # store for global metrics
        global_y_true.append(y_test_inv)
        global_y_pred.append(y_pred_inv)

        mae = mean_absolute_error(y_test_inv, y_pred_inv)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
        smape_val = smape(y_test_inv, y_pred_inv)
        r2 = r2_score(y_test_inv, y_pred_inv)
        peak_err = peak_error(y_test_inv, y_pred_inv)

        metrics_df = pd.concat([metrics_df, pd.DataFrame({
            'junction': [junction],
            'MAE': [mae],
            'RMSE': [rmse],
            'MAPE': [mape],
            'SMAPE': [smape_val],
            'R2': [r2],
            'Peak_Error': [peak_err]
        })], ignore_index=True)

        # Save model
        model.save_model(f'model_trained/LightGBM_{version}/{junction}.txt')
    # -----------------------------
    # Global metrics across all junctions
    # -----------------------------
    all_y_true = np.vstack(global_y_true)
    all_y_pred = np.vstack(global_y_pred)

    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'MAPE': mean_absolute_percentage_error(all_y_true, all_y_pred),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2': r2_score(all_y_true, all_y_pred),
        'Peak_Error': peak_error(all_y_true, all_y_pred)
    }

    print("\n✅ Global Metrics:")
    for k,v in global_metrics.items():
        print(f"{k}: {v:.4f}")
    mlflow.log_metrics(global_metrics)

# Save feature/target mapping
with open(f'model_trained/LightGBM_{version}/feature_and_target.json', 'w') as f:
    json.dump(feature_and_target, f, indent=4)

print("✅ XGBoost training completed")

Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.144887	valid_1's rmse: 0.148378
[200]	training's rmse: 0.0926397	valid_1's rmse: 0.0973174
[300]	training's rmse: 0.0637599	valid_1's rmse: 0.0688354
[400]	training's rmse: 0.0500982	valid_1's rmse: 0.0552162
[500]	training's rmse: 0.0413909	valid_1's rmse: 0.0464797
[600]	training's rmse: 0.0354732	valid_1's rmse: 0.0405495
[700]	training's rmse: 0.0308796	valid_1's rmse: 0.0358675
[800]	training's rmse: 0.027495	valid_1's rmse: 0.0324845
[900]	training's rmse: 0.0248267	valid_1's rmse: 0.0297928
[1000]	training's rmse: 0.022751	valid_1's rmse: 0.0277243
[1100]	training's rmse: 0.0211293	valid_1's rmse: 0.0261031
[1200]	training's rmse: 0.0198349	valid_1's rmse: 0.0248209
[1300]	training's rmse: 0.0186921	valid_1's rmse: 0.0236598
[1400]	training's rmse: 0.0177362	valid_1's rmse: 0.0227033
[1500]	training's rmse: 0.0168984	valid_1's rmse: 0.0218919
[1600]	training's rmse: 0.0162107	valid_1's rmse: 0

/tmp/ipykernel_4136/1409757792.py:146: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, pd.DataFrame({


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.150359	valid_1's rmse: 0.151748
[200]	training's rmse: 0.0975872	valid_1's rmse: 0.100216
[300]	training's rmse: 0.0667845	valid_1's rmse: 0.0701986
[400]	training's rmse: 0.0516372	valid_1's rmse: 0.0555259
[500]	training's rmse: 0.0416702	valid_1's rmse: 0.0457117
[600]	training's rmse: 0.0350652	valid_1's rmse: 0.0392248
[700]	training's rmse: 0.0301276	valid_1's rmse: 0.0342822
[800]	training's rmse: 0.0266377	valid_1's rmse: 0.0308288
[900]	training's rmse: 0.0239211	valid_1's rmse: 0.0281673
[1000]	training's rmse: 0.021835	valid_1's rmse: 0.0261088
[1100]	training's rmse: 0.0202622	valid_1's rmse: 0.0245822
[1200]	training's rmse: 0.018947	valid_1's rmse: 0.0233032
[1300]	training's rmse: 0.0178416	valid_1's rmse: 0.0222347
[1400]	training's rmse: 0.0169509	valid_1's rmse: 0.0213508
[1500]	training's rmse: 0.0161952	valid_1's rmse: 0.0206344
[1600]	training's rmse: 0.0155724	valid_1's rmse: 0.

FileNotFoundError: [Errno 2] No such file or directory: 'model_trained/XGBoost_0.0.2_moving_avg/feature_and_target.json'

XGBoost

In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pickle
import os
import mlflow
import json
import time

# -----------------------------
# Metrics
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = './dataset/junctions/'
version = '0.0.1'
os.makedirs(f'./model_trained/XGBoost_{version}', exist_ok=True)
mlflow.set_tracking_uri("file:model_trained/mlruns")

metrics_df = pd.DataFrame(columns=['junction', 'MAE', 'RMSE', 'MAPE', 'SMAPE', 'R2', 'Peak_Error'])
feature_and_target = {}

lag_steps = [1,2,3]

# lists to store global predictions
global_y_true, global_y_pred = [], []

with mlflow.start_run(run_name=f"XGBoost_{version}"):

    for filename in os.listdir(root)[:70]:
        df = pd.read_parquet(os.path.join(root, filename))
        junction = os.path.splitext(filename)[0]

        # -----------------------------
        # Create lag features
        # -----------------------------
        for lag in lag_steps:
            df[f'{junction}_lag{lag}'] = df[junction].shift(lag)
        df = df.fillna(0)

        features = [col for col in df.columns if col != junction]
        target = junction
        feature_and_target[junction] = {"target": target, "features": features, 'lags':lag_steps }

        # -----------------------------
        # Scaling
        # -----------------------------
        feature_scaler = MinMaxScaler()
        target_scaler = MinMaxScaler()

        df[features] = feature_scaler.fit_transform(df[features])
        df[[target]] = target_scaler.fit_transform(df[[target]])

        os.makedirs(f'model_trained/XGBoost_{version}/scalers', exist_ok=True)
        with open(f'model_trained/XGBoost_{version}/scalers/{junction}_feature_scaler.save', 'wb') as f:
            pickle.dump(feature_scaler, f)
        with open(f'model_trained/XGBoost_{version}/scalers/{junction}_target_scaler.save', 'wb') as f:
            pickle.dump(target_scaler, f)

        # -----------------------------
        # Train/test split by scenario_id
        # -----------------------------
        scenario_ids = df['scenario_id'].unique()
        split = int(0.8 * len(scenario_ids))
        train_scenarios = scenario_ids[:split]
        test_scenarios = scenario_ids[split:]

        train_df = df[df['scenario_id'].isin(train_scenarios)]
        test_df = df[df['scenario_id'].isin(test_scenarios)]

        X_train, y_train = train_df[features], train_df[target]
        X_test, y_test = test_df[features], test_df[target]

        # -----------------------------
        # Train XGBoost Regressor
        # -----------------------------
        model = xgb.XGBRegressor(
            n_estimators=5000,
            learning_rate=0.01,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            objective='reg:squarederror',
            verbosity=0
        )

        start_time = time.time()
        model.fit(
            X_train, y_train,
            eval_set=[(X_test, y_test)],
            verbose=100
        )
        elapsed_sec = time.time() - start_time
        print(f"{junction} training time: {elapsed_sec:.2f} sec")

        # -----------------------------
        # Prediction & evaluation
        # -----------------------------
        y_pred = model.predict(X_test)
        y_test_inv = target_scaler.inverse_transform(y_test.values.reshape(-1,1))
        y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))

        # store for global metrics
        global_y_true.append(y_test_inv)
        global_y_pred.append(y_pred_inv)

        mae = mean_absolute_error(y_test_inv, y_pred_inv)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
        smape_val = smape(y_test_inv, y_pred_inv)
        r2 = r2_score(y_test_inv, y_pred_inv)
        peak_err = peak_error(y_test_inv, y_pred_inv)

        metrics_df = pd.concat([metrics_df, pd.DataFrame({
            'junction': [junction],
            'MAE': [mae],
            'RMSE': [rmse],
            'MAPE': [mape],
            'SMAPE': [smape_val],
            'R2': [r2],
            'Peak_Error': [peak_err]
        })], ignore_index=True)

        # Save model
        model.save_model(f'model_trained/XGBoost_{version}/{junction}.json')

    # -----------------------------
    # Global metrics across all junctions
    # -----------------------------
    all_y_true = np.vstack(global_y_true)
    all_y_pred = np.vstack(global_y_pred)

    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'MAPE': mean_absolute_percentage_error(all_y_true, all_y_pred),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2': r2_score(all_y_true, all_y_pred),
        'Peak_Error': peak_error(all_y_true, all_y_pred)
    }

    print("\n✅ Global Metrics:")
    for k,v in global_metrics.items():
        print(f"{k}: {v:.4f}")
    mlflow.log_metrics(global_metrics)

# Save feature/target mapping
with open(f'model_trained/XGBoost_{version}/feature_and_target.json', 'w') as f:
    json.dump(feature_and_target, f, indent=4)

print("✅ XGBoost training completed")


[0]	validation_0-rmse:0.25956
[100]	validation_0-rmse:0.24196
[200]	validation_0-rmse:0.23707
[300]	validation_0-rmse:0.23595
[400]	validation_0-rmse:0.23553
[500]	validation_0-rmse:0.23557
[600]	validation_0-rmse:0.23562
[700]	validation_0-rmse:0.23572
[800]	validation_0-rmse:0.23584
[900]	validation_0-rmse:0.23603
[1000]	validation_0-rmse:0.23627
[1100]	validation_0-rmse:0.23658
[1200]	validation_0-rmse:0.23676
[1300]	validation_0-rmse:0.23706
[1400]	validation_0-rmse:0.23728
[1500]	validation_0-rmse:0.23749
[1600]	validation_0-rmse:0.23775
[1700]	validation_0-rmse:0.23790
[1800]	validation_0-rmse:0.23809
[1900]	validation_0-rmse:0.23832
[2000]	validation_0-rmse:0.23854
[2100]	validation_0-rmse:0.23879
[2200]	validation_0-rmse:0.23904
[2300]	validation_0-rmse:0.23924
[2400]	validation_0-rmse:0.23943
[2500]	validation_0-rmse:0.23967
[2600]	validation_0-rmse:0.23981
[2700]	validation_0-rmse:0.23996
[2800]	validation_0-rmse:0.24010
[2900]	validation_0-rmse:0.24031
[3000]	validation_0-rm

KeyboardInterrupt: 

Prophet


In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
import pickle
import mlflow
import json
import time

# -----------------------------
# Metrics
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = './dataset/junctions/'
version = '0.0.1'
os.makedirs(f'./model_trained/Prophet_{version}', exist_ok=True)
mlflow.set_tracking_uri("file:model_trained/mlruns")

metrics_df = pd.DataFrame(columns=['junction', 'MAE', 'RMSE', 'MAPE', 'SMAPE', 'R2', 'Peak_Error'])
feature_and_target = {}

global_y_true, global_y_pred = [], []

with mlflow.start_run(run_name=f"Prophet_{version}"):

    for filename in os.listdir(root)[:70]:
        df = pd.read_parquet(os.path.join(root, filename))
        junction = os.path.splitext(filename)[0]

        # -----------------------------
        # Scenario-based train/test split
        # -----------------------------
        scenario_ids = df['scenario_id'].unique()
        split_idx = int(0.8 * len(scenario_ids))
        train_scenarios = scenario_ids[:split_idx]
        test_scenarios = scenario_ids[split_idx:]

        train_df = df[df['scenario_id'].isin(train_scenarios)]
        test_df = df[df['scenario_id'].isin(test_scenarios)]

        # Prepare for Prophet
        train_prophet = train_df[['time_id', junction]].rename(columns={'time_id': 'ds', junction: 'y'})
        test_prophet = test_df[['time_id', junction]].rename(columns={'time_id': 'ds', junction: 'y'})
        train_prophet['y'] = train_prophet['y'].fillna(method='ffill')
        test_prophet['y'] = test_prophet['y'].fillna(method='ffill')

        feature_and_target[junction] = {"target": junction}

        # -----------------------------
        # Train Prophet
        # -----------------------------
        model = Prophet(daily_seasonality=False)
        start_time = time.time()
        model.fit(train_prophet)
        elapsed_sec = time.time() - start_time
        print(f"{junction} training time: {elapsed_sec:.2f} sec")

        # -----------------------------
        # Forecast test scenarios
        # -----------------------------
        future = test_prophet[['ds']]
        forecast = model.predict(future)
        y_pred = forecast['yhat'].values
        y_true = test_prophet['y'].values

        global_y_true.append(y_true)
        global_y_pred.append(y_pred)

        # -----------------------------
        # Metrics per junction
        # -----------------------------
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mape = mean_absolute_percentage_error(y_true, y_pred)
        smape_val = smape(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        peak_err = peak_error(y_true, y_pred)

        metrics_df = pd.concat([metrics_df, pd.DataFrame({
            'junction': [junction],
            'MAE': [mae],
            'RMSE': [rmse],
            'MAPE': [mape],
            'SMAPE': [smape_val],
            'R2': [r2],
            'Peak_Error': [peak_err]
        })], ignore_index=True)

        # Save model
        with open(f'model_trained/Prophet_{version}/{junction}.pkl', 'wb') as f:
            pickle.dump(model,f)

    # -----------------------------
    # Global metrics
    # -----------------------------
    all_y_true = np.concatenate(global_y_true)
    all_y_pred = np.concatenate(global_y_pred)

    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'MAPE': mean_absolute_percentage_error(all_y_true, all_y_pred),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2': r2_score(all_y_true, all_y_pred),
        'Peak_Error': peak_error(all_y_true, all_y_pred)
    }

    print("\n✅ Global Metrics:")
    for k,v in global_metrics.items():
        print(f"{k}: {v:.4f}")
    mlflow.log_metrics(global_metrics)

# Save feature/target mapping
with open(f'model_trained/Prophet_{version}/feature_and_target.json', 'w') as f:
    json.dump(feature_and_target, f, indent=4)

print("✅ Prophet training completed")


/tmp/ipykernel_13570/3798897201.py:63: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_prophet['y'] = train_prophet['y'].fillna(method='ffill')
/tmp/ipykernel_13570/3798897201.py:64: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_prophet['y'] = test_prophet['y'].fillna(method='ffill')
21:15:45 - cmdstanpy - INFO - Chain [1] start processing
21:15:48 - cmdstanpy - INFO - Chain [1] done processing


J434 training time: 3.29 sec


/tmp/ipykernel_13570/3798897201.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, pd.DataFrame({


AttributeError: 'Prophet' object has no attribute 'save'